In [1]:
import os
import cv2
import random
import rasterio
import numpy as np
from PIL import Image, ImageEnhance
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import box
from rasterio.windows import Window

In [4]:
def save_raster(output_raster, data, profile):
    """
    Save raster data while keeping metadata.
    """
    with rasterio.open(output_raster, "w", **profile) as dest:
        for i in range(data.shape[0]):
            dest.write(data[i], i + 1)

def rotateAndDeleteEmptySpace(fileName, outputName, degreeOfRotation):
    # Open the raster .tif file
    with rasterio.open(fileName) as src:
        # Read the data from the raster (shape will be (bands, height, width))
        data = src.read()

        # Get the metadata and affine transformation
        profile = src.profile
        transform = src.transform

        cropped_bands = []
        for i in range(data.shape[0]):  # Loop over each band
            band = data[i]  # Get the i-th band
            pil_band = Image.fromarray(band)
            rotated_band = pil_band.rotate(degreeOfRotation, expand=True)
            rotated_band_data = np.array(rotated_band)
            
            # Crop empty space
            non_empty_pixels = rotated_band_data > 0
            non_empty_rows = np.any(non_empty_pixels, axis=1)
            non_empty_cols = np.any(non_empty_pixels, axis=0)
            
            min_row, max_row = np.where(non_empty_rows)[0][[0, -1]]
            min_col, max_col = np.where(non_empty_cols)[0][[0, -1]]
            
            cropped_band_data = rotated_band_data[min_row:max_row+1, min_col:max_col+1]
            cropped_bands.append(cropped_band_data)

        # Stack the cropped bands into a multi-band image
        cropped_data = np.stack(cropped_bands)

        # Update the profile and save the multi-band image
        profile.update(width=cropped_data.shape[2], height=cropped_data.shape[1])

        save_raster(outputName, cropped_data, profile)

def flip_raster(input_raster, output_raster):
    """
    Flip raster image horizontally.
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read()

        # Flip all bands
        flipped_data = np.array([np.fliplr(data[i]) for i in range(data.shape[0])])
        # flipped_data = np.array([np.flipud(data[i]) for i in range(data.shape[0])])

        save_raster(output_raster, flipped_data, profile)
    
def crop_raster(input_raster, output_raster, offsetRatio):
    """
    Crop a multilayer raster file to the specified bounds.
    
    Parameters:
    - input_raster: Path to the input raster file.
    - output_raster: Path to save the cropped raster file.
    - offsetRatio: crop ratio
    """
    with rasterio.open(input_raster) as src:
        # Create a geometry for the crop bounds
        bounds = src.bounds
        xOffset = (bounds.top - bounds.bottom) * offsetRatio
        yOffset = (bounds.right - bounds.left) * offsetRatio
        crop_bounds = (bounds.left + yOffset, bounds.bottom + xOffset, 
                       bounds.right - yOffset, bounds.top - xOffset)
        crop_geom = [box(*crop_bounds)]

        # Crop the raster using the geometry
        out_image, out_transform = mask(src, crop_geom, crop=True)

        # Update metadata for the new cropped raster
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "dtype": src.dtypes[0],  # Preserve original dtype
            "nodata": src.nodata,  # Preserve nodata value
            "compress": src.profile.get("compress", "LZW"),  # Preserve compression
            "photometric": src.tags().get("photometric", "RGB"),  # Preserve color interpretation
        })

        save_raster(output_raster, out_image, out_meta)

def rescale_raster(input_raster, output_raster, scale_factor):
    """
    Rescale raster image (Zoom In & Out).
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read()

        new_h, new_w = int(data.shape[1] * scale_factor), int(data.shape[2] * scale_factor)

        # Resize each band
        rescaled_data = np.array([cv2.resize(data[i], (new_w, new_h), interpolation=cv2.INTER_LINEAR) for i in range(data.shape[0])])

        # Crop back to original size if zoomed in
        if scale_factor > 1:
            start_h, start_w = (new_h - data.shape[1]) // 2, (new_w - data.shape[2]) // 2
            rescaled_data = rescaled_data[:, start_h:start_h + data.shape[1], start_w:start_w + data.shape[2]]

        profile.update(height=data.shape[1], width=data.shape[2])

        save_raster(output_raster, rescaled_data, profile)

def getDSMFilePath(DSMMainPath):
    ''' 
    get tall the dsm file path as list
    '''
    returnData = []

    # check if the data path exist
    if(not(os.path.exists(DSMMainPath))):
        print(DSMMainPath, "path not exist")
        return DSMMainPath
    
    # loop into each main picture (day)
    for folder in os.listdir(DSMMainPath):
        folderPath = DSMMainPath + "/" + folder

        # loop into each label
        for subFolder in os.listdir(folderPath):
            subFolderPath = folderPath + "/" + subFolder

            # loop into each file
            for targetFile in os.listdir(subFolderPath):
                if(targetFile[:3] == "DSM" and targetFile[-12:] == "original.tif"):
                    targetFilePath = subFolderPath + "/" + targetFile
                    returnData.append(targetFilePath)

    return returnData

def createNewFolder(targetFilePath, foldername):
    ''' 
    create new folder for enhanced files
    '''
    prevFolderNameList = []
    
    # loop in to every target path
    for file in targetFilePath:

        # get only previouse folder path, preparing for create new folder
        prevFolderName = file.split('/')[:-1]
        prevFolderName = '/'.join(prevFolderName)

        # get rid off same files in same folder
        if(prevFolderName not in prevFolderNameList):
            prevFolderNameList.append(prevFolderName)
            
            # name new folder
            newFolderName = prevFolderName + '/' + foldername

            # check if the folder already exist
            if(not(os.path.exists(newFolderName))):
                # make new folder
                os.mkdir(newFolderName)
                print("created folder: ", newFolderName)
            else:
                print("folder already exist")

def loopAugmentDSM(dsmPathList):
    ''' 
    loop into eact dsm file for augmenttaion, save file to "Augmented" folder
    '''

    count = 0
    # loop into every dsm file
    for targetFilePath in dsmPathList:

        # create output path in Enhanced folder
        splittedPath = targetFilePath.split('/')
        prevFolderPath = splittedPath[:-1]
        outFilePath = '/'.join(prevFolderPath) + "/" + "Augmented" + "/" + splittedPath[-1][:-4]

        # correct tilt the original dsm
        # create out path for corrected tilt one outside the augment folder
        outFilePathCorrectedDegree = '/'.join(prevFolderPath) + "/" + splittedPath[-1][:-4] + "_correctedTilt.tif"
        # check if file not already exist
        if(not(os.path.exists(outFilePathCorrectedDegree))):
            rotateAndDeleteEmptySpace(targetFilePath, outFilePathCorrectedDegree, 103)
        else:
            print("tilt corrected file already exist")

        # rotated, flipped, zoomed
        # create out file path for rotated
        outFilePathRotated = outFilePath + "_rotated.tif"
        # check if file not already exist
        if(not(os.path.exists(outFilePathRotated))):
            rotateAndDeleteEmptySpace(outFilePathCorrectedDegree, outFilePathRotated, 180)
        else:
            print("rotated file already exist")

        # create out file path for flipped
        outFilePathFlipped = outFilePath + "_flipped.tif"
        # check if file not already exist
        if(not(os.path.exists(outFilePathFlipped))):
            flip_raster(outFilePathCorrectedDegree, outFilePathFlipped)
        else:
            print("flipped file already exist")

        # create out file path for zoomed
        outFilePathZoomed = outFilePath + "_zoomed.tif"
        # check if file not already exist
        if(not(os.path.exists(outFilePathZoomed))):
            crop_raster(outFilePathCorrectedDegree, outFilePathZoomed, 0.1)
        else:
            print("zoomed file already exist")

        count += 1
        print(f"finished {count} / {len(dsmPathList)} : {count*100/(len(dsmPathList)):.2f}%")

def recursiveGetRidofauxxml(folderPath):
    ''' 
    get rid of every .aux file in folder
    '''
    # loop into each file
    for file in os.listdir(folderPath):
        # assign file path
        filePath = folderPath + "/" + file
        # check if file is .aux
        if(file[-8:] == ".aux.xml"):
            # remove .aux file
            os.remove(filePath)
            print(f"removed {file}")
        
        # check if folder and call this function recursively
        if(os.path.isdir(filePath)):
            recursiveGetRidofauxxml(filePath)

In [ ]:
dsmMainPath = "D:/ice-wheat/data/dataForProcess/mainData/DSM"
dsmPathList = getDSMFilePath(dsmMainPath)
loopAugmentDSM(dsmPathList)



tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 1 / 3565 : 0.03%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 2 / 3565 : 0.06%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 3 / 3565 : 0.08%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 4 / 3565 : 0.11%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 5 / 3565 : 0.14%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 6 / 3565 : 0.17%
tilt corrected file already exist
rotated file already exist
flipped file already exist
zoomed file already exist
finished 7 / 3565 : 0.20%
tilt corrected file 

In [5]:
recursiveGetRidofauxxml("/Volumes/HD-PCFSU3-A/ice-wheat/data/dataForProcess/mainData")

removed DSM_202402131116_7_original_correctedTilt.tif.aux.xml
removed RGB_202402131116_7_original.jpg.aux.xml
